In [1]:
"""
1) Прочитайте главы 2-3 из книги "Spark: The Definitive Guide"
2) Установите библиотеки numpy, pandas, matplotlib и seasborn.
Для этого введите в консоли:

source ~/Work/bin/activate
pip3 install numpy
pip3 install pandas
pip3 install matplotlib
pip3 install seaborn

3) Скачайте датасет TikTok_songs_2022.csv по ссылке https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2022
4) Выполните задания 1-5 с помощью SQL и методов DataFrame.
5) Ответьте на вопросы в задании 6.
"""


'\n1) Прочитайте главы 2-3 из книги "Spark: The Definitive Guide"\n2) Установите библиотеки numpy, pandas, matplotlib и seasborn.\nДля этого введите в консоли:\n\nsource ~/Work/bin/activate\npip3 install numpy\npip3 install pandas\npip3 install matplotlib\npip3 install seaborn\n\n3) Скачайте датасет TikTok_songs_2022.csv по ссылке https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2022\n4) Выполните задания 1-5 с помощью SQL и методов DataFrame.\n5) Ответьте на вопросы в задании 6.\n'

In [2]:
#!pip install numpy pandas matplotlib seaborn


In [3]:
# Создаем SparkSession
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName("SparkApp").getOrCreate()


22/09/15 00:09:39 WARN Utils: Your hostname, student-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/09/15 00:09:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/15 00:09:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Загружаем датасет https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2022
tiktokData2022 = (
    spark.read.option("inferSchema", "true")
    .option("header", "true")
    .csv("TikTok_songs_2022.csv")
)

tiktokData2022.createOrReplaceTempView("TikTok_songs_2022")


In [5]:
# 0: Получите таблицу из  5 названий треков с их исполнителями.

# +--------------------+-------------+
# |          track_name|  artist_name|
# +--------------------+-------------+
# |Running Up That H...|    Kate Bush|
# |           As It Was| Harry Styles|
# |             Sunroof|  Nicky Youre|
# |          Heat Waves|Glass Animals|
# |     About Damn Time|        Lizzo|
# +--------------------+-------------+

from pyspark.sql.functions import desc

sqlWay = spark.sql(
    """
SELECT track_name,artist_name
FROM TikTok_songs_2022
LIMIT 5
"""
)

dataFrameWay = tiktokData2022.select("track_name", "artist_name").limit(5)

sqlWay.show()
dataFrameWay.show()


+--------------------+-------------+
|          track_name|  artist_name|
+--------------------+-------------+
|Running Up That H...|    Kate Bush|
|           As It Was| Harry Styles|
|             Sunroof|  Nicky Youre|
|          Heat Waves|Glass Animals|
|     About Damn Time|        Lizzo|
+--------------------+-------------+

+--------------------+-------------+
|          track_name|  artist_name|
+--------------------+-------------+
|Running Up That H...|    Kate Bush|
|           As It Was| Harry Styles|
|             Sunroof|  Nicky Youre|
|          Heat Waves|Glass Animals|
|     About Damn Time|        Lizzo|
+--------------------+-------------+



In [6]:
# 1: Получите список исполнителей с популярностью больше 90%

# ['Drake', 'Harry Styles', 'The Weeknd']

sqlWay = spark.sql(
    """
SELECT DISTINCT artist_name FROM TikTok_songs_2022
WHERE artist_pop > 90
"""
)

dataFrameWay = (
    tiktokData2022.select("artist_name")
    .where(tiktokData2022.artist_pop > 90)
    .distinct()
)

print(sqlWay.rdd.map(lambda x: x[0]).collect())
print(dataFrameWay.rdd.map(lambda x: x[0]).collect())


['Drake', 'Harry Styles', 'The Weeknd']
['Drake', 'Harry Styles', 'The Weeknd']


In [7]:
# 2: Выведите список исполнителей и количество их треков в порядке уменьшения.
# Ограничьте выборку исполнителями, у которых не менее 3 треков в наборе данных.

# +--------------+----------------+
# |   artist_name|number_of_tracks|
# +--------------+----------------+
# |      Doja Cat|               8|
# |        Coopex|               6|
# |     Dame Dame|               5|
# |     Lil Nas X|               4|
# |Alex Alexander|               4|
# |       YES YES|               4|
# |         Farux|               4|
# | Ariana Grande|               3|
# |       Astelle|               3|
# |  Alex D'Rosso|               3|
# |      Dua Lipa|               3|
# |    The Weeknd|               3|
# |     Nito-Onna|               3|
# |     Pop Smoke|               3|
# | The Kid LAROI|               3|
# +--------------+----------------+

from pyspark.sql.functions import desc, count

sqlWay = spark.sql(
    """
SELECT artist_name, count(*) as number_of_tracks
FROM TikTok_songs_2022
GROUP BY artist_name
HAVING number_of_tracks >= 3
ORDER BY number_of_tracks DESC
"""
)

dataFrameWay = (
    tiktokData2022.groupBy("artist_name")
    .agg(count("*").alias("number_of_tracks"))
    .where("number_of_tracks >= 3")
    .orderBy("number_of_tracks", ascending=False)
)

sqlWay.show()
dataFrameWay.show()


+--------------+----------------+
|   artist_name|number_of_tracks|
+--------------+----------------+
|      Doja Cat|               8|
|        Coopex|               6|
|     Dame Dame|               5|
|     Lil Nas X|               4|
|Alex Alexander|               4|
|       YES YES|               4|
|         Farux|               4|
| Ariana Grande|               3|
|       Astelle|               3|
|  Alex D'Rosso|               3|
|      Dua Lipa|               3|
|    The Weeknd|               3|
|     Nito-Onna|               3|
|     Pop Smoke|               3|
| The Kid LAROI|               3|
+--------------+----------------+

+--------------+----------------+
|   artist_name|number_of_tracks|
+--------------+----------------+
|      Doja Cat|               8|
|        Coopex|               6|
|     Dame Dame|               5|
|     Lil Nas X|               4|
|Alex Alexander|               4|
|       YES YES|               4|
|         Farux|               4|
| Ariana Gran

In [8]:
# 3: Выведите список названий треков, средняя громкость которых лежит в диапазоне от -10 до -9 Дб.

# ['Glimpse of Us', 'Left and Right (Charlie Puth) [feat. Jung Kook of BTS] - Sped Up Version',
# 'Frozen', "Can't Get You Out of My Head", 'I Hate U', 'I AM WOMAN', 'Buss It', 'Whatever You Like',
# 'MONEY', 'Freaks', 'telepatía', 'Falling Down', 'Party Girl', 'Skechers', 'Good Looking']

sqlWay = spark.sql(
    """
SELECT track_name FROM TikTok_songs_2022
WHERE loudness BETWEEN -10 AND -9
"""
)

dataFrameWay = tiktokData2022.select("track_name").where(
    tiktokData2022.loudness.between(-10, -9)
)

print(sqlWay.rdd.map(lambda x: x[0]).collect())
print(dataFrameWay.rdd.map(lambda x: x[0]).collect())


['Glimpse of Us', 'Left and Right (Charlie Puth) [feat. Jung Kook of BTS] - Sped Up Version', 'Frozen', "Can't Get You Out of My Head", 'I Hate U', 'I AM WOMAN', 'Buss It', 'Whatever You Like', 'MONEY', 'Freaks', 'telepatía', 'Falling Down', 'Party Girl', 'Skechers', 'Good Looking']
['Glimpse of Us', 'Left and Right (Charlie Puth) [feat. Jung Kook of BTS] - Sped Up Version', 'Frozen', "Can't Get You Out of My Head", 'I Hate U', 'I AM WOMAN', 'Buss It', 'Whatever You Like', 'MONEY', 'Freaks', 'telepatía', 'Falling Down', 'Party Girl', 'Skechers', 'Good Looking']


In [9]:
# 4: Выведите список треков, в названиях которых есть слово "love" (без учета регистра).

# ['Another Love', 'love nwantiti (ah ah ah)', 'Tainted Love', 'Looking for Love', 'Free Falling Love',
# 'Electric Love', 'Feel Your Love', 'What You Know Bout Love', 'Savage Love (Laxed - Siren Beat)',
# 'Love Me Again', 'Lovely']

from pyspark.sql.functions import lower

sqlWay = spark.sql(
    """
SELECT track_name FROM TikTok_songs_2022
WHERE LOWER(track_name) LIKE '%love%'
"""
)

dataFrameWay = tiktokData2022.select("track_name").where(
    lower(tiktokData2022.track_name).like("%love%")
)

print(sqlWay.rdd.map(lambda x: x[0]).collect())
print(dataFrameWay.rdd.map(lambda x: x[0]).collect())


['Another Love', 'love nwantiti (ah ah ah)', 'Tainted Love', 'Looking for Love', 'Free Falling Love', 'Electric Love', 'Feel Your Love', 'What You Know Bout Love', 'Savage Love (Laxed - Siren Beat)', 'Love Me Again', 'Lovely']
['Another Love', 'love nwantiti (ah ah ah)', 'Tainted Love', 'Looking for Love', 'Free Falling Love', 'Electric Love', 'Feel Your Love', 'What You Know Bout Love', 'Savage Love (Laxed - Siren Beat)', 'Love Me Again', 'Lovely']


In [10]:
# 5: Получите таблицу из песен Арианы Гранде и Astelle. Отсортируйте их по примеру ниже.

# +--------------------+-------------+
# |          track_name|  artist_name|
# +--------------------+-------------+
# |               34+35|Ariana Grande|
# |motive (with Doja...|Ariana Grande|
# |           positions|Ariana Grande|
# |             Dreamer|      Astelle|
# |           K Boy Bye|      Astelle|
# |                Paid|      Astelle|
# +--------------------+-------------+

sqlWay = spark.sql(
    """
SELECT track_name, artist_name
FROM TikTok_songs_2022
WHERE artist_name in ("Ariana Grande", "Astelle")
ORDER BY artist_name, album
"""
)

dataFrameWay = (
    tiktokData2022.select("track_name", "artist_name")
    .where(tiktokData2022.artist_name.isin("Ariana Grande", "Astelle"))
    .orderBy("artist_name", "track_name")
)

sqlWay.show()
dataFrameWay.show()


+--------------------+-------------+
|          track_name|  artist_name|
+--------------------+-------------+
|               34+35|Ariana Grande|
|motive (with Doja...|Ariana Grande|
|           positions|Ariana Grande|
|             Dreamer|      Astelle|
|           K Boy Bye|      Astelle|
|                Paid|      Astelle|
+--------------------+-------------+

+--------------------+-------------+
|          track_name|  artist_name|
+--------------------+-------------+
|               34+35|Ariana Grande|
|motive (with Doja...|Ariana Grande|
|           positions|Ariana Grande|
|             Dreamer|      Astelle|
|           K Boy Bye|      Astelle|
|                Paid|      Astelle|
+--------------------+-------------+



In [11]:
# 6: Какую задачу решает приведенный ниже код?
# Ответ: Разделяет все треки на 20 групп по их параметрам
# В чем суть метода K средних?
# В пространстве признаков случайно выбираются N центров кластеров
# (Объект будет относится к группе того кластера к центру которого он находится ближе всего)
# Итеративно центры кластеров смещаются в сторону центра массы относящихся к ним в данный момент объектов
# Процесс прекращается когда их смещение становится меньше eps
# В результате получаются готовые кластеры
# Какие еще существуют методы машинного обучения для решения данной задачи?
# Affinity propagation, Сдвиг среднего значения, Спектральная кластеризация и другие
# Какие выводы можно сделать о полученных группах?
# Трудно ответить на этот вопрос
# Интуитивно возникает гипотиза о том что треки в пределах одной группы должны быть похожи между собой
# Однако анализ показал что треки относящиеся к альбомам из которых представлено >= 3 трека оказались распределены по разным группам
# Planet Her - 4 трека в 3 группах
# Shoot For The Stars Aim For The Moon - 3 трека в 3 группах
# То же можно сказать о треках относящихся к одному исполнителю если рассматривать тех у которых >= 5 треков
# Doja Cat - 8 треков, 5 групп
# Coopex - 6 треков, 5 групп
# Dame Dame - 5 треков, 4 группы
# Прослушивание треков из самых двух самых маленьких групп так же не привело к какому-то пониманию
# В группе 13 - 2 трека:
# Xilo - Aesthetic - это Хип-хоп/рэп
# Peachy! - Falling for U - это R&B/соул
# единственное что в них похоже - фоновая музыка
# В группе 9 - 3 трека:
# Freaks - Surf Curse - это Инди-поп
# Write This Down (Instrumental) - SoulChef - это Инструментальный хип-хоп
# Astronomia - Vicetone - танцевальная музыка(та самая)
# как и в жанрах на слух какая-либо близость между ними не обнаружена

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.sql import Row
from pyspark.ml.feature import MinMaxScaler
from IPython.display import display
import numpy as np

k = 20

np.set_printoptions(precision=3, suppress=True)

columns_to_scale = [
    "danceability",
    "energy",
    "speechiness",
    "acousticness",
    "instrumentalness",
    "liveness",
    "valence",
    "duration_ms",
    "tempo",
    "track_pop",
]
assemblers = [
    VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_scale
]
scalers = [
    MinMaxScaler(inputCol=col + "_vec", outputCol=col + "_scaled")
    for col in columns_to_scale
]

vectorAssembler = (
    VectorAssembler()
    .setInputCols(
        [
            "danceability_scaled",
            "energy_scaled",
            "speechiness_scaled",
            "acousticness_scaled",
            "instrumentalness_scaled",
            "liveness_scaled",
            "valence_scaled",
            "duration_ms_scaled",
            "tempo_scaled",
            "track_pop_scaled",
        ]
    )
    .setOutputCol("features")
)

transformationPipeline = Pipeline().setStages(assemblers + scalers + [vectorAssembler])

fittedPipeline = transformationPipeline.fit(tiktokData2022)
transformedDF = fittedPipeline.transform(tiktokData2022)

kmeans = KMeans().setK(k).setSeed(300)

kmModel = kmeans.fit(transformedDF)

df_pred = kmModel.transform(transformedDF)

centers = kmModel.clusterCenters()
for i in range(0, k):
    print("group " + str(i))
    print(centers[i])
    df = (
        df_pred.selectExpr(
            "track_name",
            "artist_name",
            "album",
            "danceability",
            "energy",
            "speechiness",
            "acousticness",
            "instrumentalness",
            "liveness",
            "valence",
            "duration_ms",
            "tempo",
            "track_pop",
        )
        .where("prediction = " + str(i))
        .toPandas()
    )
    display(df)


group 0
[0.745 0.779 0.497 0.064 0.001 0.139 0.551 0.327 0.455 0.582]


track_name    artist_name  \
0                        Riptide  Viva La Panda   
1               When You're Gone       Paradigm   
2              THATS WHAT I WANT      Lil Nas X   
3                        We Know   JAM & Philly   
4            When You Were Young          Amero   
5   Bundles (feat. Taylor Girlz)   Kayla Nicole   
6                           Paid        Astelle   
7                             Up        Cardi B   
8   Best Friend (feat. Doja Cat)       Saweetie   
9                       Rude Boy        YES YES   
10                           Fly   Alex D'Rosso   
11                      The Boss         Coopex   
12           Cool for the Summer   Alex D'Rosso   
13                    Boss Bitch       Doja Cat   
14                        Tap In       Saweetie   
15                           BOP         DaBaby   

                                      album  danceability  energy  \
0                                   Riptide         0.621   0.847   
1                          When You're Gone         0.783   0.753   
2                                   MONTERO         0.737   0.846   
3                                   We Know         0.871   0.819   
4                       When You Were Young         0.806   0.845   
5              Bundles (feat. Taylor Girlz)         0.939   0.682   
6                                      Paid         0.861   0.631   
7                         Up (Instrumental)         0.868   0.795   
8   Best Friend (feat. Doja Cat) [Remix EP]         0.840   0.766   
9                                  Rude Boy         0.739   0.734   
10                                      Fly         0.676   0.842   
11                                 The Boss         0.734   0.789   
12                      Cool for the Summer         0.805   0.871   
13                               Boss Bitch         0.707   0.955   
14                                   Tap In         0.954   0.696   
15                                     KIRK         0.769   0.787   

    speechiness  acousticness  instrumentalness  liveness  valence  \
0         0.278       0.05840          0.000001    0.1090    0.373   
1         0.325       0.01170          0.000000    0.1390    0.663   
2         0.220       0.00614          0.000000    0.0486    0.546   
3         0.256       0.14200          0.000000    0.1060    0.712   
4         0.190       0.08250          0.000000    0.2600    0.508   
5         0.306       0.00152          0.000000    0.0552    0.584   
6         0.230       0.01880          0.000283    0.0954    0.675   
7         0.269       0.00120          0.019300    0.0461    0.819   
8         0.136       0.00302          0.000004    0.0684    0.402   
9         0.259       0.02420          0.000000    0.0921    0.471   
10        0.239       0.05780          0.001370    0.2090    0.388   
11        0.243       0.08090          0.000115    0.2150    0.258   
12        0.286       0.17700          0.000008    0.2660    0.443   
13        0.222       0.12700          0.000000    0.2020    0.575   
14        0.204       0.00576          0.000498    0.0754    0.432   
15        0.367       0.18900          0.000000    0.1290    0.836   

    duration_ms    tempo  track_pop  
0        172431  101.996         51  
1        145161  124.036         53  
2        143901   87.981         88  
3        198982  112.114         39  
4        128095  125.998         31  
5        154806  130.022         67  
6        180923  117.993         39  
7        156945  166.000         70  
8        155884   94.018         45  
9        174858  101.995         50  
10       134857  140.142         53  
11       133810  125.992         47  
12       171696  136.010         49  
13       134240  125.989         78  
14       139413  100.036         67  
15       159715  126.770         76

group 1
[0.59  0.768 0.107 0.077 0.003 0.204 0.433 0.351 0.515 0.547]


track_name       artist_name  \
0                 Flex On My Ex (with Bankrol Hayden)          KillBunk   
1                                          That Bxtch     Taylor Watson   
2                                     Warning / Signs    Matvey Emerson   
3                                            Serenity              Shab   
4                                        Tainted Love          Thatsimo   
5                                               Crazy      Alex D'Rosso   
6                                              Dinero  Trinidad Cardona   
7                                         Ocean Drive          Harddope   
8                                         Young Folks    Jungle Jonsson   
9                                         Complicated         Nito-Onna   
10                             Astronaut In The Ocean       Masked Wolf   
11                                                Why              Padé   
12                                     Heart of Glass         Nito-Onna   
13                                       Mr. Saxobeat         Nito-Onna   
14                                    Pumped Up Kicks            Madism   
15                      Ready 4 Luv - TMLH Summer Mix          CAZZETTE   
16  Wellerman - Sea Shanty / 220 KID x Billen Ted ...      Nathan Evans   
17                                          Follow Me            Senhit   
18                                        Wicked Game           YES YES   
19                                          K Boy Bye           Astelle   
20                                       Play With Me         Ana Shine   
21                                            Forever      Nadia Gattas   
22                                     Feel Your Love        Yusuf Alev   
23                                       Say It Right            Coopex   
24                                 One too Many Times               JRL   
25                                            Forever        Gianni Blu   
26                                         What's up?            LeXxìv   
27                                         Let Her Go             LANDR   

                                                album  danceability  energy  \
0                 Flex On My Ex (with Bankrol Hayden)         0.619   0.707   
1                                          That Bxtch         0.645   0.681   
2                                     Warning / Signs         0.712   0.829   
3                                            Serenity         0.688   0.595   
4                                        Tainted Love         0.646   0.765   
5                                               Crazy         0.681   0.865   
6                                              Dinero         0.708   0.782   
7                                         Ocean Drive         0.645   0.868   
8                                         Young Folks         0.750   0.731   
9                                         Complicated         0.724   0.692   
10                             Astronaut In The Ocean         0.778   0.695   
11                               Why (with WLHELMINA)         0.610   0.902   
12                                     Heart of Glass         0.750   0.838   
13                                       Mr. Saxobeat         0.696   0.899   
14                                    Pumped Up Kicks         0.669   0.687   
15                      Ready 4 Luv (TMLH Summer Mix)         0.679   0.869   
16  Wellerman (Sea Shanty / 220 KID x Billen Ted R...         0.722   0.893   
17                                          Follow Me         0.707   0.867   
18                                        Wicked Game         0.755   0.835   
19                                          K Boy Bye         0.547   0.865   
20                                       Play With Me         0.714   0.666   
21                                            Forever         0.750   0.879   
22                                     Feel Your L

group 2
[0.8   0.408 0.213 0.317 0.018 0.348 0.78  0.429 0.294 0.774]


...

track_name        artist_name  \
0  Left and Right (Charlie Puth) [feat. Jung Kook...  sped up nightcore   
1                                 Pope Is a Rockstar              SALES   
2                                           I Hate U                SZA   
3                                       Running Away          VANO 3000   
4                                 Just The Two of Us            Kauai45   
5                                         Party Girl     StaySolidRocky   
6                                              Renee              SALES   

                                               album  danceability  energy  \
0  Left and Right (Charlie Puth) [feat. Jung Kook...         0.827   0.456   
1                                           SALES LP         0.725   0.446   
2                                           I Hate U         0.535   0.388   
3                                       Running Away         0.580   0.696   
4                                 Just The Two of Us         0.592   0.214   
5                                         Party Girl         0.728   0.431   
6                                              Renee         0.924   0.386   

   speechiness  acousticness  instrumentalness  liveness  valence  \
0       0.0942         0.622          0.000098    0.1090    0.689   
1       0.0287         0.850          0.243000    0.1070    0.311   
2       0.1550         0.507          0.000000    0.1100    0.412   
3       0.0408         0.660          0.852000    0.1450    0.721   
4       0.1150         0.936          0.000016    0.2530    0.422   
5       0.0622         0.749          0.000000    0.0996    0.629   
6       0.0548         0.755          0.002530    0.1180    0.424   

   duration_ms    tempo  track_pop  
0       135978  114.269         58  
1       187333   90.024         74  
2       174000  106.703         79  
3       111787   93.688         72  
4       117037   81.672         67  
5       147800  130.022         71  
6       179125  108.957         65

group 18
[0.564 0.769 0.106 0.187 0.015 0.273 0.146 0.338 0.462 0.534]


track_name    artist_name                         album  \
0                      Baby Girl    Disco Lines                     Baby Girl   
1                          Pepas        YES YES                         Pepas   
2                     Ocean Eyes          Farux                    Ocean Eyes   
3   Can't Get You Out of My Head        loafers  Can't Get You Out of My Head   
4                         Pieces      Nippandab                        Pieces   
5                 Paint It Black      Dame Dame                Paint It Black   
6              Free Falling Love  Lucas Estrada             Free Falling Love   
7                   Say It Right          MØRFI                  Say It Right   
8                          Creep        Naeleck                         Creep   
9                      Apologize          Farux                     Apologize   
10                   Lost on You      sad girls                   Lost on You   
11                 Glad You Came       Braaheim                 Glad You Came   
12                   Without You      Baby Bluu                   Without You   
13                  Falling Down    Andrew Nagy                  Falling Down   
14                     Only Girl         Coopex                     Only Girl   
15                     Old Times       Xtinctor                     Old Times   
16              Hey, Soul Sister          LANDR              Hey, Soul Sister   
17                        Demons            CPX                        Demons   
18                      Rockstar   Ilkay Sencan                      Rockstar   
19                        Lovely         Alfons                        Lovely   

    danceability  energy  speechiness  acousticness  instrumentalness  \
0          0.730   0.907       0.0606        0.0921          0.114000   
1          0.691   0.823       0.0626        0.0235          0.000000   
2          0.729   0.763       0.0345        0.6680          0.026400   
3          0.642   0.610       0.0992        0.1600          0.000007   
4          0.698   0.780       0.0626        0.1570          0.032500   
5          0.690   0.741       0.0363        0.0433          0.004290   
6          0.771   0.932       0.0424        0.3110          0.000158   
7          0.764   0.793       0.0709        0.0132          0.000980   
8          0.615   0.829       0.0421        0.0555          0.001560   
9          0.677   0.710       0.0552        0.5320          0.000000   
10         0.599   0.692       0.0739        0.1240          0.000000   
11         0.732   0.919       0.1150        0.0271          0.000866   
12         0.625   0.698       0.1460        0.0107          0.000000   
13         0.574   0.852       0.0438        0.0320          0.070500   
14         0.792   0.832       0.0397        0.3670          0.001640   
15         0.581   0.718       0.0851        0.0195          0.000000   
16         0.642   0.725       0.0571        0.1920          0.000000   
17         0.700   0.848       0.2170        0.3170          0.000000   
18         0.771   0.826       0.0604        0.1420          0.030000   
19         0.504   0.677       0.0640        0.3160          0.001270   

    liveness  valence  duration_ms    tempo  track_pop  
0     0.2300   0.2490       111074  120.968         76  
1     0.0781   0.1490       134330  125.973         58  
2     0.1470   0.1130       137029  125.989         42  
3     0.4080   0.2940       168000  129.886         52  
4     0.3090   0.2030       177956  122.076         42  
5     0.0975   0.0774       146190  125.985         56  
6     0.1160   0.1390       155000  120.012         58  
7     0.3190   0.1270       129677  124.084         50  
8     0.4260   0.0754       203226  124.013         65  
9     0.3370   0.3250       129410  126.109         48  
10    0.3750   0.3070       144349  121.796         53  
11    0.3750   0.1360       190879  125.997         48  
12    0.1560   0.2040       132625  120.119  

group 19
[0.649 0.515 0.619 0.119 0.004 0.169 0.491 0.453 0.789 0.741]


track_name          artist_name  \
0                          TO THE MOON             Jnr Choi   
1                            Long Road        HotBlock Jmoe   
2                         Light Switch         Charlie Puth   
3                           I AM WOMAN            Emmy Meli   
4                                MONEY                 LISA   
5                            So Pretty        Reyanna Maria   
6             When Hello Means Goodbye       Alex Alexander   
7                                Hawái               Maluma   
8   Gravity (feat. Tyler, The Creator)         Brent Faiyaz   
9      WAP (feat. Megan Thee Stallion)              Cardi B   
10        Mood Swings (feat. Lil Tjay)            Pop Smoke   
11                              Savage  Megan Thee Stallion   
12          Life Is Good (feat. Drake)               Future   
13         OUT WEST (feat. Young Thug)             JACKBOYS   

                                   album  danceability  energy  speechiness  \
0                            TO THE MOON         0.745   0.650        0.346   
1                              Long Road         0.738   0.685        0.409   
2            Light Switch (Instrumental)         0.690   0.626        0.306   
3                             I AM WOMAN         0.646   0.471        0.157   
4                                 LALISA         0.831   0.554        0.218   
5                              So Pretty         0.799   0.543        0.240   
6                              On My Way         0.741   0.475        0.205   
7                           PAPI JUANCHO         0.783   0.727        0.389   
8     Gravity (feat. Tyler, The Creator)         0.539   0.615        0.252   
9        WAP (feat. Megan Thee Stallion)         0.935   0.454        0.375   
10  Shoot For The Stars Aim For The Moon         0.480   0.602        0.368   
11                                  Suga         0.843   0.741        0.334   
12            Life Is Good (feat. Drake)         0.676   0.609        0.481   
13                              JACKBOYS         0.802   0.591        0.225   

    acousticness  instrumentalness  liveness  valence  duration_ms    tempo  \
0         0.0451          0.007580    0.1110    0.386       152137  144.047   
1         0.1350          0.000000    0.4000    0.663       204000  162.039   
2         0.1050          0.000335    0.0881    0.912       185681  184.009   
3         0.1220          0.000000    0.1250    0.455       232813  170.084   
4         0.1610          0.000061    0.1520    0.396       168228  140.026   
5         0.0512          0.042500    0.1340    0.250        85742  187.906   
6         0.0214          0.000000    0.3510    0.597       158160  166.016   
7         0.1850          0.000025    0.0626    0.558       199112  180.067   
8         0.1730          0.005600    0.1440    0.493       214693  163.924   
9         0.0194          0.000000    0.0824    0.357       187541  133.073   
10        0.4870          0.000000    0.0848    0.269       213333  179.798   
11        0.0252          0.000000    0.0960    0.680       155497  168.983   
12        0.0706          0.000000    0.1520    0.508       237735  142.037   
13        0.0104          0.000000    0.1960    0.309       157712  139.864   

    track_pop  
0          79  
1          52  
2          79  
3          76  
4          79  
5          66  
6          44  
7          77  
8          72  
9          80  
10         78  
11         70  
12         77  
13         77